In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

In [3]:
from load_data.data_loader import load_df
df = load_df()
df.head()

,problem_class,problem_score,score_norm,combined_text,constraint,math,algoword,length
0,hard,2500.0,0.629630,Title: Digits Description: John gave Jack a ve...,5.301030,12,0,888
1,medium,2000.0,0.444444,Title: Neural Network country Description: Due...,2.025306,13,2,995
2,medium,2100.0,0.481481,Title: Property Description: Bill is a famous ...,4.698970,70,0,1292
3,medium,2100.0,0.481481,Title: Exploration plan Description: The compe...,4.301030,3,0,1253
4,medium,2100.0,0.481481,Title: Casinos and travel Description: John ha...,5.000000,4,2,1337


In [4]:
feature_cols = ["constraint", "math", "algoword", "length"]

X_text = df["combined_text"]
X_num  = df[feature_cols].fillna(0)
y = df["problem_class"]

(
    X_text_train, X_text_test,
    X_num_train,  X_num_test,
    y_train,  y_test,
) = train_test_split(
    X_text, X_num, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1, 2), min_df=3, max_df=0.9)

X_text_train_vec = tfidf.fit_transform(X_text_train)
X_text_test_vec  = tfidf.transform(X_text_test)

X_num_train_s = StandardScaler().fit_transform(X_num_train)
X_num_test_s  = StandardScaler().fit_transform(X_num_test)

X_train_final = hstack([X_text_train_vec, X_num_train_s])
X_test_final  = hstack([X_text_test_vec,  X_num_test_s])


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

svm_clf = LinearSVC(class_weight="balanced", max_iter=5000)

svm_clf.fit(X_train_final, y_train)

y_pred_svm = svm_clf.predict(X_test_final)

print("SVC")
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))


Linear SVM
Accuracy: 0.5075648414985591
              precision    recall  f1-score   support

        easy       0.59      0.63      0.61       885
        hard       0.55      0.56      0.55       980
      medium       0.36      0.33      0.35       911

    accuracy                           0.51      2776
   macro avg       0.50      0.51      0.50      2776
weighted avg       0.50      0.51      0.50      2776

